# Pokročilé SQL
#### Struktura a optimalizace

# Testovací databáze

Nástroj `pgbench` přímo s Postgresem.

In [ ]:
!echo "CREATE DATABASE dbtest;" | psql postgres
!pgbench -i -s 10 dbtest

!pgbench -T 30 -c 4 dbtest

In [22]:
import sqlalchemy
import pandas

eng = sqlalchemy.create_engine("postgresql:///dbtest")

def explain(query, extra=[]):
    keywords = ["EXPLAIN"] + extra
    for line, in eng.execute(" ".join(keywords) + " " + query):
        print(line)
        
def explain_analyze(q):
    return explain(q, ["ANALYZE"])
        
def sql(query):
    return pandas.read_sql_query(query, eng)

%load_ext sql
%sql postgresql:///dbtest
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.style = "PLAIN_COLUMNS"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
!echo "\d pgbench_accounts" | psql dbtest
!echo
!echo "\d pgbench_history" | psql dbtest

              Table "public.pgbench_accounts"
  Column  |     Type      | Collation | Nullable | Default 
----------+---------------+-----------+----------+---------
 aid      | integer       |           | not null | 
 bid      | integer       |           |          | 
 abalance | integer       |           |          | 
 filler   | character(84) |           |          | 
Indexes:
    "pgbench_accounts_pkey" PRIMARY KEY, btree (aid)


                    Table "public.pgbench_history"
 Column |            Type             | Collation | Nullable | Default 
--------+-----------------------------+-----------+----------+---------
 tid    | integer                     |           |          | 
 bid    | integer                     |           |          | 
 aid    | integer                     |           |          | 
 delta  | integer                     |           |          | 
 mtime  | timestamp without time zone |           |          | 
 filler | character(22)               |        

In [3]:
!echo "\d pgbench_branches" | psql dbtest
!echo
!echo "\d pgbench_tellers" | psql dbtest

              Table "public.pgbench_branches"
  Column  |     Type      | Collation | Nullable | Default 
----------+---------------+-----------+----------+---------
 bid      | integer       |           | not null | 
 bbalance | integer       |           |          | 
 filler   | character(88) |           |          | 
Indexes:
    "pgbench_branches_pkey" PRIMARY KEY, btree (bid)


              Table "public.pgbench_tellers"
  Column  |     Type      | Collation | Nullable | Default 
----------+---------------+-----------+----------+---------
 tid      | integer       |           | not null | 
 bid      | integer       |           |          | 
 tbalance | integer       |           |          | 
 filler   | character(84) |           |          | 
Indexes:
    "pgbench_tellers_pkey" PRIMARY KEY, btree (tid)



# Sequential scan

Jde řádek po řádku přes celou tabulku.

In [23]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts WHERE abalance > 0

for row in result.iloc[:,0]: print(row)

 * postgresql:///dbtest
Seq Scan on pgbench_accounts  (cost=0.00..29171.00 rows=125271 width=97)
  Filter: (abalance > 0)


In [5]:
df = %sql SELECT * FROM pgbench_accounts WHERE abalance > 0
df.head()

 * postgresql:///dbtest


,aid,bid,abalance,filler
0,50,1,3382,...
1,58,1,679,...
2,27,1,4816,...
3,55,1,2010,...
4,45,1,2044,...


# Explain vs Explain Analyze

In [52]:
r = %sql EXPLAIN SELECT * FROM pgbench_accounts WHERE abalance > 0
for row in r.iloc[:,0]: print(row)


 * postgresql:///dbtest
Seq Scan on pgbench_accounts  (cost=0.00..29171.00 rows=125271 width=97)
  Filter: (abalance > 0)


In [53]:
r = %sql EXPLAIN ANALYZE SELECT * FROM pgbench_accounts WHERE abalance > 0
for row in r.iloc[:,0]: print(row)

 * postgresql:///dbtest
Seq Scan on pgbench_accounts  (cost=0.00..29171.00 rows=125271 width=97) (actual time=0.015..102.513 rows=122394 loops=1)
  Filter: (abalance > 0)
  Rows Removed by Filter: 877606
Planning time: 0.057 ms
Execution time: 106.263 ms


# View

* Jen definice SQL dotazu
* Čitelnost
* Dotaz `SELECT x FROM myview WHERE y` se přeloží na

```sql
SELECT x
FROM ( .... definice view ... ) myview
WHERE y
```

In [75]:
%sql CREATE OR REPLACE VIEW positive_accounts AS SELECT * FROM pgbench_accounts WHERE abalance > 0;

result = %sql EXPLAIN ANALYZE SELECT * FROM positive_accounts
for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
 * postgresql:///dbtest
Seq Scan on pgbench_accounts  (cost=0.00..29171.00 rows=125271 width=97) (actual time=0.012..77.139 rows=122394 loops=1)
  Filter: (abalance > 0)
  Rows Removed by Filter: 877606
Planning time: 0.059 ms
Execution time: 80.214 ms


# Index nad jedním sloupcem

* B-tree (default)
    * binární vyhledávací strom
* hash
    * hash tabulka, tj. v Pythonu `dict`
* GIST
    * "plugin" indexy - např. PostGIS přidává indexy pro geografická data
* SP-GIST
    * Další speciální indexy (prohledávání prostoru, trie)
* GIN
    * Invertované indexy.
        * Záznam běžného indexu: dokument 1 obsahuje slova "šel", "Honza", "buchty"
        * Záznam invertovaného indexu: slovo "buchty" je v dokumentech 1, 78 a 222

In [24]:
%%sql

DROP TABLE IF EXISTS pgbench_accounts_indexed;
CREATE TEMPORARY TABLE pgbench_accounts_indexed AS SELECT * FROM pgbench_accounts;

 * postgresql:///dbtest


""


In [25]:
%%time
%sql CREATE INDEX pgbench_accounts_indexed_abalance ON pgbench_accounts_indexed (abalance);

 * postgresql:///dbtest
CPU times: user 3.91 ms, sys: 0 ns, total: 3.91 ms
Wall time: 561 ms


""


In [26]:
result = %sql EXPLAIN ANALYZE SELECT * FROM pgbench_accounts_indexed WHERE abalance > 0

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Bitmap Heap Scan on pgbench_accounts_indexed  (cost=3590.26..24150.92 rows=333333 width=352) (actual time=12.980..74.161 rows=122394 loops=1)
  Recheck Cond: (abalance > 0)
  Heap Blocks: exact=16383
  ->  Bitmap Index Scan on pgbench_accounts_indexed_abalance  (cost=0.00..3506.92 rows=333333 width=0) (actual time=10.759..10.759 rows=122394 loops=1)
        Index Cond: (abalance > 0)
Planning time: 0.287 ms
Execution time: 77.481 ms


In [41]:
result = %sql EXPLAIN ANALYZE SELECT * FROM pgbench_accounts_indexed WHERE abalance > 999999999
for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Bitmap Heap Scan on pgbench_accounts_indexed  (cost=3590.26..24150.92 rows=333333 width=352) (actual time=0.013..0.014 rows=0 loops=1)
  Recheck Cond: (abalance > 999999999)
  ->  Bitmap Index Scan on pgbench_accounts_indexed_abalance  (cost=0.00..3506.92 rows=333333 width=0) (actual time=0.012..0.012 rows=0 loops=1)
        Index Cond: (abalance > 999999999)
Planning time: 0.056 ms
Execution time: 0.037 ms


In [40]:
result = %sql EXPLAIN ANALYZE SELECT * FROM pgbench_accounts WHERE abalance > 999999999
for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Gather  (cost=1000.00..22881.73 rows=24 width=97) (actual time=45.239..49.543 rows=0 loops=1)
  Workers Planned: 2
  Workers Launched: 2
  ->  Parallel Seq Scan on pgbench_accounts  (cost=0.00..21879.33 rows=10 width=97) (actual time=43.068..43.068 rows=0 loops=3)
        Filter: (abalance > 999999999)
        Rows Removed by Filter: 333333
Planning time: 0.072 ms
Execution time: 49.570 ms


# Optimalizátor

* Počítá si statistiky
    * Příkaz `ANALYZE` nebo periodicky na pozadí s `VACUUM` podle nastavení databáze.
* Dokáže spojit více kroků nebo rozdělit práci.
* Ne vždy mu dojde, že dělá nesmysl.

_Připomínka:_ 
```sql
CREATE VIEW positive_accounts AS 
SELECT * FROM pgbench_accounts WHERE abalance > 0
```

In [36]:
explain("SELECT * FROM positive_accounts WHERE abalance < 0")

Gather  (cost=1000.00..24421.00 rows=5000 width=97)
  Workers Planned: 2
  ->  Parallel Seq Scan on pgbench_accounts  (cost=0.00..22921.00 rows=2083 width=97)
        Filter: ((abalance > 0) AND (abalance < 0))


In [61]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed WHERE abalance < 0 AND abalance > 0
for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Bitmap Heap Scan on pgbench_accounts_indexed  (cost=67.08..4691.75 rows=5000 width=352)
  Recheck Cond: ((abalance < 0) AND (abalance > 0))
  ->  Bitmap Index Scan on pgbench_accounts_indexed_abalance  (cost=0.00..65.83 rows=5000 width=0)
        Index Cond: ((abalance < 0) AND (abalance > 0))


# Index Scan vs Bitmap Index Scan

* Optimalizátor se rozhoduje, co kdy zvolí.

* Část řádků je potřeba odfiltrovat, ale hodně zbyde => Bitmap Index Scan
* Zaměření na málo konkrétních řádků => Index Scan (typicky dotaz na řádek podle unikátního klíče).

In [62]:
result = %sql EXPLAIN ANALYZE SELECT * FROM pgbench_accounts WHERE aid = 445

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Index Scan using pgbench_accounts_pkey on pgbench_accounts  (cost=0.42..2.64 rows=1 width=97) (actual time=0.017..0.018 rows=1 loops=1)
  Index Cond: (aid = 445)
Planning time: 0.071 ms
Execution time: 0.038 ms


# Joiny

* Optimalizátoru je jedno, jestli píšete
```
FROM a, b, c
WHERE a.id = b.aid AND c.bid = b.id
```
* Nebo
```
FROM a
    INNER JOIN b ON a.id = b.aid
    INNER JOIN c ON c.bid = b.id
```


## Na pořadí JOINů může a taky nemusí záležet.

* `join_collapse_limit`
* `SET join_collapse_limit = x` jej nastaví pro aktuální spojení.
* Větší hodnota znamená delší plánování, ale potenciálně lepší výsledek.
     * Ale jestli se databáze u vašeho dotazu zasekne už na `EXPLAIN` (ne `EXPLAIN ANALYZE`), možná je moc nastavený moc vysoko.

In [66]:
%sql SHOW join_collapse_limit

 * postgresql:///dbtest


,join_collapse_limit
0,8


In [69]:
%sql SET join_collapse_limit = 10
%sql SHOW join_collapse_limit

 * postgresql:///dbtest
 * postgresql:///dbtest


,join_collapse_limit
0,10


In [70]:
%%sql
CREATE OR REPLACE VIEW transactions AS 
SELECT aid, tid, pgbench_branches.bid, tbalance, delta, mtime, abalance
    FROM pgbench_history
        LEFT JOIN pgbench_tellers USING (tid)
        LEFT JOIN pgbench_accounts USING (aid)
        LEFT JOIN pgbench_branches ON pgbench_history.bid = pgbench_branches.bid

 * postgresql:///dbtest


""


In [73]:
explain("SELECT * FROM transactions")

Hash Left Join  (cost=39198.47..46334.62 rows=280131 width=32)
  Hash Cond: (pgbench_history.bid = pgbench_branches.bid)
  ->  Hash Left Join  (cost=39197.25..45286.41 rows=280131 width=32)
        Hash Cond: (pgbench_history.aid = pgbench_accounts.aid)
        ->  Hash Left Join  (cost=26.25..5380.07 rows=280131 width=28)
              Hash Cond: (pgbench_history.tid = pgbench_tellers.tid)
              ->  Seq Scan on pgbench_history  (cost=0.00..4587.31 rows=280131 width=24)
              ->  Hash  (cost=25.00..25.00 rows=100 width=8)
                    ->  Seq Scan on pgbench_tellers  (cost=0.00..25.00 rows=100 width=8)
        ->  Hash  (cost=26671.00..26671.00 rows=1000000 width=8)
              ->  Seq Scan on pgbench_accounts  (cost=0.00..26671.00 rows=1000000 width=8)
  ->  Hash  (cost=1.10..1.10 rows=10 width=4)
        ->  Seq Scan on pgbench_branches  (cost=0.00..1.10 rows=10 width=4)


In [72]:
explain_analyze("SELECT tid FROM transactions")

Seq Scan on pgbench_history  (cost=0.00..4587.31 rows=280131 width=4) (actual time=0.020..24.504 rows=280131 loops=1)
Planning time: 0.215 ms
Execution time: 31.925 ms


# Funkce

* Čitelnost
* Méně chyb
    * Lépe se testuje
    * Definice logiky na jednom místě

Mimochodem: [pgTAP: unit testing v Postgresu](https://pgtap.org/)

In [98]:
%%sql
CREATE OR REPLACE FUNCTION has_money(in balance numeric) RETURNS varchar AS $$ 
    SELECT CASE WHEN balance <= 0 THEN 'no chance'
                WHEN balance > 10000 THEN 'oh boy!'
                ELSE 'boring'
            END;
    $$ LANGUAGE SQL;

SELECT val, has_money(val) FROM (VALUES (-5), (887), (1000000)) vals(val);

 * postgresql:///dbtest


,val,has_money
0,-5,no chance
1,887,boring
2,1000000,oh boy!


In [100]:
result = %sql EXPLAIN SELECT * FROM positive_accounts WHERE has_money(abalance) = 'oh boy!';

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Seq Scan on pgbench_accounts  (cost=0.00..36671.00 rows=626 width=97)
  Filter: ((abalance > 0) AND ((CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END)::text = 'oh boy!'::text))


## Nápovědy pro optimalizátor

Funkce může být označena jako

* `IMMUTABLE` - funkce vždy vrátí stejnou hodnotu, když ji zavolám se stejnými parametry
    * Např. naše `has_money`
* `STABLE` - skoro IMMUTABLE, ale hodnota se může měnit mezi SQL dotazy.
    * Např. závisí na nastavení databáze, dělá dotaz do nějaké tabulky.
* `VOLATILE` - default, může si dělat cokoliv a nebude optimalizátorem zjednodušena, ani vyhozena

Další nápovědy:

* `COST` - pro ne-SQL funknce odhadnutí jak složité je spočítat jedno zavolání (v relativních jednotkách)
* `ROWS` - pro funkce vracející řádky - kolik řádků zhruba vrátí

# Složitější indexy

* Přes více sloupců
    * Vždy jen v rámci jedné tabulky
        * Ale `MATERIALIZED VIEW` je jedna tabulka
        * `TEMPORARY TABLE AS (SELECT ...)` je také jedna tabulka

In [150]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts JOIN pgbench_branches USING(bid) WHERE bbalance < -50000 AND aid > 720001

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Hash Join  (cost=1.57..12353.82 rows=55964 width=457)
  Hash Cond: (pgbench_accounts.bid = pgbench_branches.bid)
  ->  Index Scan using pgbench_accounts_pkey on pgbench_accounts  (cost=0.42..11306.85 rows=279819 width=97)
        Index Cond: (aid > 720001)
  ->  Hash  (cost=1.12..1.12 rows=2 width=364)
        ->  Seq Scan on pgbench_branches  (cost=0.00..1.12 rows=2 width=364)
              Filter: (bbalance < '-50000'::integer)


### Optimalizátor si zkusí poskládat několik existujících indexů

In [165]:
%%sql
CREATE INDEX IF NOT EXISTS pgbench_accounts_indexed_aid ON pgbench_accounts_indexed (aid);
CREATE INDEX IF NOT EXISTS pgbench_accounts_indexed_bid ON pgbench_accounts_indexed (bid);

 * postgresql:///dbtest


""


In [166]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed JOIN pgbench_branches USING(bid) WHERE bbalance < -50000 AND aid > 720001

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Nested Loop  (cost=53.74..8351.82 rows=3333 width=712)
  ->  Seq Scan on pgbench_branches  (cost=0.00..1.12 rows=2 width=364)
        Filter: (bbalance < '-50000'::integer)
  ->  Bitmap Heap Scan on pgbench_accounts_indexed  (cost=53.74..4158.68 rows=1667 width=352)
        Recheck Cond: (bid = pgbench_branches.bid)
        Filter: (aid > 720001)
        ->  Bitmap Index Scan on pgbench_accounts_indexed_bid  (cost=0.00..53.33 rows=5000 width=0)
              Index Cond: (bid = pgbench_branches.bid)


### Nebo může použít složený index

In [167]:
%%sql
DROP INDEX IF EXISTS pgbench_accounts_indexed_aid;
DROP INDEX IF EXISTS pgbench_accounts_indexed_bid;
CREATE INDEX IF NOT EXISTS pgbench_accounts_indexed_aid_bid ON pgbench_accounts_indexed (aid, bid);

 * postgresql:///dbtest


""


In [168]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed JOIN pgbench_branches USING(bid) WHERE bbalance < -50000 AND aid > 720001

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Nested Loop  (cost=0.42..11575.31 rows=3333 width=712)
  ->  Seq Scan on pgbench_branches  (cost=0.00..1.12 rows=2 width=364)
        Filter: (bbalance < '-50000'::integer)
  ->  Index Scan using pgbench_accounts_indexed_aid_bid on pgbench_accounts_indexed  (cost=0.42..5770.43 rows=1667 width=352)
        Index Cond: ((aid > 720001) AND (bid = pgbench_branches.bid))


### Nebo použije začátek složeného indexu

In [172]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed JOIN pgbench_branches USING(bid) WHERE bbalance < 500000 AND aid > 720001

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Hash Join  (cost=3591.48..25045.69 rows=13333 width=712)
  Hash Cond: (pgbench_accounts_indexed.bid = pgbench_branches.bid)
  ->  Bitmap Heap Scan on pgbench_accounts_indexed  (cost=3590.26..24150.92 rows=333333 width=352)
        Recheck Cond: (aid > 720001)
        ->  Bitmap Index Scan on pgbench_accounts_indexed_aid_bid  (cost=0.00..3506.92 rows=333333 width=0)
              Index Cond: (aid > 720001)
  ->  Hash  (cost=1.12..1.12 rows=8 width=364)
        ->  Seq Scan on pgbench_branches  (cost=0.00..1.12 rows=8 width=364)
              Filter: (bbalance < 500000)


Bitmap Index Scan on pgbench_accounts_indexed_aid_bid

Index Cond: (aid > 720001)

### Ale složený index nemůže použít od konce
(B-tree index je strom)

In [173]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed WHERE bid < 10

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Seq Scan on pgbench_accounts_indexed  (cost=0.00..28894.00 rows=333333 width=352)
  Filter: (bid < 10)


# Složitější indexy
* Přes výraz nebo volání funkce


In [179]:
%sql DROP INDEX IF EXISTS pgbench_accounts_indexed_function
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed WHERE has_money(abalance) = 'oh boy!'

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
 * postgresql:///dbtest
Seq Scan on pgbench_accounts_indexed  (cost=0.00..33894.00 rows=5000 width=352)
  Filter: ((CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END)::text = 'oh boy!'::text)


In [180]:
%sql CREATE INDEX pgbench_accounts_indexed_function ON pgbench_accounts_indexed (has_money(abalance))

 * postgresql:///dbtest


""


In [182]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed WHERE has_money(abalance) = 'oh boy!'

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Bitmap Heap Scan on pgbench_accounts_indexed  (cost=60.08..4697.25 rows=5000 width=352)
  Recheck Cond: (CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END = 'oh boy!'::text)
  ->  Bitmap Index Scan on pgbench_accounts_indexed_function  (cost=0.00..58.83 rows=5000 width=0)
        Index Cond: (CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END = 'oh boy!'::text)


### Optimalizátor nepracuje nutně s funkcí, ale s výrezem, kterým je definována

In [184]:
result = %sql EXPLAIN SELECT * FROM pgbench_accounts_indexed WHERE ((CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END)::text = 'oh boy!'::text)

for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Bitmap Heap Scan on pgbench_accounts_indexed  (cost=60.08..4697.25 rows=5000 width=352)
  Recheck Cond: (CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END = 'oh boy!'::text)
  ->  Bitmap Index Scan on pgbench_accounts_indexed_function  (cost=0.00..58.83 rows=5000 width=0)
        Index Cond: (CASE WHEN (abalance <= 0) THEN 'no chance'::text WHEN (abalance > 10000) THEN 'oh boy!'::text ELSE 'boring'::text END = 'oh boy!'::text)


# Paralelismus

In [42]:
result = %sql EXPLAIN ANALYZE SELECT COUNT(*) FROM pgbench_accounts WHERE abalance > 999999999
for row in result["QUERY PLAN"]:
    print(row)

 * postgresql:///dbtest
Finalize Aggregate  (cost=22879.57..22879.58 rows=1 width=8) (actual time=43.819..43.819 rows=1 loops=1)
  ->  Gather  (cost=22879.36..22879.57 rows=2 width=8) (actual time=43.765..48.247 rows=3 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial Aggregate  (cost=21879.36..21879.37 rows=1 width=8) (actual time=41.217..41.218 rows=1 loops=3)
              ->  Parallel Seq Scan on pgbench_accounts  (cost=0.00..21879.33 rows=10 width=0) (actual time=41.213..41.213 rows=0 loops=3)
                    Filter: (abalance > 999999999)
                    Rows Removed by Filter: 333333
Planning time: 0.071 ms
Execution time: 48.293 ms


# Problémy, které jsme měli    

* Dotaz, který dříve fungoval dobře je najednou ukrutně pomalý. Změnil se query plan.
    * Tabulka se změnila, ale Postgres nestihl ještě sám udělat `ANALYZE`.

* V EXPLAIN se objeví **drahá** "nested loop".
    * Může znamenat, že optimalizátor už neví kudy kam.
    * Problém byl v tom, že někdy má `ANALYZE` problém odhadnout distribuci hodnot.
        * Tabulka předchozích nákupů - mnoho opakování.

* Chybějící indexy

* Výpočet na několik kroků
    * Optimalizace vizualizace grafů v adminovi
    * Nejdřív spočítat matici spojení, pak ji projít a vytvořit graf.
    * Mezivýsledek se uloží do dočasné tabulky a naindexuje se.

# Strukturování kódu

* Skládejte z menších celků
    * VIEW, FUNCTION
        * Optimalizátor si poradí
        * Když si neporadí, uvidíte drahou část `EXPLAIN`
    * Pozor na `WITH` - zlepšuje čitelnost, ale většinou je "optimalizační bariérou"

* Testujte menší celky
* Konzistentní pojmenování
    * Jedno jméno proměnné má vždy znamenat to stejné
    * Jedna věc by se (ideálně) měla vždy jmenovat stejně
* Automatické testy?

# Čtení složitého `EXPLAIN`

1. Najít drahé operace
2. Rozdělit na části
3. Představit si, jak by se dal výsledek spočítat